In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found

In [2]:
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)

from peft import get_peft_model, LoraConfig
import pandas as pd
from huggingface_hub import login
from torch import float16

2025-05-24 16:51:36.690414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748105496.877655      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748105496.932937      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct" # for LLaMA 3.1
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"     # for Mistral

In [4]:
train_df = pd.read_csv("/kaggle/input/summarization/cnn_dailymail/train.csv", engine="python", on_bad_lines='skip', nrows=1000)
test_df = pd.read_csv("/kaggle/input/summarization/cnn_dailymail/test.csv", engine="python", on_bad_lines='skip')

In [5]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
login() #login using hf token with access to llama and mistral models

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  
    bnb_4bit_compute_dtype=float16, 
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [11]:
# Apply LoRA
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

In [12]:
def preprocess_function(example):
    instruction = "### Instruction:\nSummarize the following article."
    input_text = f"\n\n### Input:\n{example['article']}"
    response = f"\n\n### Response:\n{example['highlights']}{tokenizer.eos_token}"

    full_prompt = instruction + input_text + response

    tokenized = tokenizer(
        full_prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None
    )

    # Identify the start index of the response for label masking
    response_start = full_prompt.index("### Response:\n") + len("### Response:\n")
    response_token_start = len(tokenizer(full_prompt[:response_start], truncation=True, max_length=512)["input_ids"])

    # Initialize labels with -100
    labels = [-100] * len(tokenized["input_ids"])
    labels[response_token_start:] = tokenized["input_ids"][response_token_start:]
    tokenized["labels"] = labels

    return tokenized

In [13]:
train_tokenized = train_dataset.map(preprocess_function, remove_columns=train_dataset.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to='none',
    logging_steps=1,
    label_names=["labels"]
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
)

In [17]:
trainer.train()

Step,Training Loss
1,5.081700
2,2.954900
3,2.120900
4,2.346100
5,1.849300
6,1.861000
7,1.282100
8,1.870200
9,1.583800
10,1.533400


TrainOutput(global_step=62, training_loss=1.4887217115009985, metrics={'train_runtime': 4515.6604, 'train_samples_per_second': 0.443, 'train_steps_per_second': 0.014, 'total_flos': 4.289621454225408e+16, 'train_loss': 1.4887217115009985, 'epoch': 1.96})

In [18]:
model.save_pretrained("fine-tuned-model")
tokenizer.save_pretrained("fine-tuned-model")

('fine-tuned-model/tokenizer_config.json',
 'fine-tuned-model/special_tokens_map.json',
 'fine-tuned-model/tokenizer.model',
 'fine-tuned-model/added_tokens.json',
 'fine-tuned-model/tokenizer.json')

In [30]:
# Interactive loop
print("Enter an article or paragraph to summarize (type 'exit' to quit):")

user_input = input("\nArticle: ").strip()

prompt = f"### Instruction:\nSummarize the following article.\n\n### Input:\n{user_input}\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
    )

generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
summary = generated.split("### Response:")[-1].strip()

print(f"\nSummary:\n{summary}")

Enter an article or paragraph to summarize (type 'exit' to quit):



Article:  Bruce Jenner will break his silence in a two-hour interview with Diane Sawyer later this month. The former Olympian and reality TV star, 65, will speak in a 'far-ranging' interview with Sawyer for a special edition of '20/20' on Friday April 24, ABC News announced on Monday. The interview comes amid growing speculation about the father-of-six's transition to a woman, and follows closely behind his involvement in a deadly car crash in California in February. And while the Kardashian women are known for enjoying center stage, they will not be stealing Bruce's spotlight because they will be in Armenia when the interview airs, according to TMZ. Scroll down for video . Speaking out: Bruce Jenner, pictured on 'Keeping Up with the Kardashians' will speak out in a 'far-ranging' interview with Diane Sawyer later this month, ABC News announced on Monday . Return: Diane Sawyer, who recently mourned the loss of her husband, will return to ABC for the interview . Rumors started swirling 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Summary:
Bruce Jenner will speak out in a 'far-ranging' interview with Diane Sawyer later this month .
The interview comes amid growing speculation about the father-of-six's transition to a woman .
Jenner, 65, will speak out in a 'far-ranging' interview with Diane Sawyer later this month .
The interview comes amid growing speculation about the father-of-six's transition to a woman .


In [28]:
print(test_df['article'][4],end="\n\n")
print(test_df['highlights'][4])

Bruce Jenner will break his silence in a two-hour interview with Diane Sawyer later this month. The former Olympian and reality TV star, 65, will speak in a 'far-ranging' interview with Sawyer for a special edition of '20/20' on Friday April 24, ABC News announced on Monday. The interview comes amid growing speculation about the father-of-six's transition to a woman, and follows closely behind his involvement in a deadly car crash in California in February. And while the Kardashian women are known for enjoying center stage, they will not be stealing Bruce's spotlight because they will be in Armenia when the interview airs, according to TMZ. Scroll down for video . Speaking out: Bruce Jenner, pictured on 'Keeping Up with the Kardashians' will speak out in a 'far-ranging' interview with Diane Sawyer later this month, ABC News announced on Monday . Return: Diane Sawyer, who recently mourned the loss of her husband, will return to ABC for the interview . Rumors started swirling around Jenn